In [1]:
import os
import rasterio as rio
from torch.utils.data import Dataset
import torchvision
import torch
import numpy as np

C:\Users\aanwar\Anaconda3\envs\py38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class SenDataset(Dataset):
    
    
    # dirc represent the folder path 
    def __init__(self, dirc, transform = None):
        self.dirc = dirc
        self.transform = transform
        
        #get the directory for image and masks
        #self.img= os.listdir(dirc)
        
        self.image_dir = os.path.join(self.dirc , "image")
        self.mask_dir = os.path.join(self.dirc ,"masks")

        self.images = os.listdir(self.image_dir)
        self.masks = os.listdir(self.mask_dir)

    
    def __len__(self):
        x = len(self.images)
        return x
    
    
    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir,self.images[index])
        mask_path1 = os.path.join(self.mask_dir,self.images[index].replace ('S2.tiff','mask_aguada.tif'))
        mask_path2 = os.path.join(self.mask_dir,self.images[index].replace ('S2.tiff','mask_building.tif'))
        mask_path3 = os.path.join(self.mask_dir,self.images[index].replace ('S2.tiff','mask_platform.tif'))
        
        #print (mask_path1)

        image = rio.open(img_path)
        mask1 = rio.open(mask_path1)
        mask2 = rio.open(mask_path2)
        mask3 = rio.open(mask_path3)
        
        #return numpy values
        image = image.read()
        
        # change it to 0 ansd 1
        mask1 = mask1.read()/255
        mask2= mask2.read()/255
        mask3 = mask3.read()/255
        
        #stack the mask in one file
        stack_mask = np.zeros(mask1.shape[0:3],np.float32)
        
        if mask1 is not None:  stack_mask[ mask1  == 1 ] = 1    
        if mask2 is not None:  stack_mask[ mask2  == 1 ] = 2 
        if mask3 is not None:  stack_mask[ mask3 == 1] = 3   

            
        tens_image = torch.from_numpy(image).to("cpu")
        tens_mask = torch.from_numpy(stack_mask).to("cpu")
        
        
        #apply trasformation
        if self.transform:
            tens_image = self.transform(tens_image)
            tens_mask = self.transform(tens_mask)
        
        return tens_image, tens_mask
    
    
    #selects a random image index for debugging.
    def choose(self): 
        return self.randint(0,len(self))
        